In [1]:
from psruq.source.table_utils import (
    collect_scores_into_dict_miss,
    extract_same_different_dataframes,
    aggregate_over_measures,
)
import pandas as pd
from psruq.source.path_config import REPOSITORY_ROOT
from source.losses.constants import LossName
from source.metrics.constants import GName
from source.datasets.constants import DatasetName
from source.models.constants import ModelSource

import numpy as np
import os
from IPython.display import display

pd.set_option("display.max_rows", None)

stty: 'standard input': Inappropriate ioctl for device


In [2]:
ind_dataset = DatasetName.CIFAR10_NOISY_LABEL.value
model_source = ModelSource.OUR_MODELS.value

full_mis_rocauc = pd.read_pickle(
    f"{REPOSITORY_ROOT}/tables/central_tables/final/{ind_dataset}_{model_source}_full_mis_rocauc.pkl"
)

In [3]:
full_mis_rocauc.sample(10)

,UQMetric,LossFunction,RocAucScores_array,architecture,training_dataset,base_rule,RiskType
15,LogScore TotalRisk inner central,CrossEntropy,"[0.6912572280870063, 0.8144304905229434, 0.816...",resnet18,cifar10_noisy_label,LogScore,inner central
190,SphericalScore TotalRisk outer outer,BrierScore,"[0.8226008000593636, 0.7914094185461955, 0.752...",resnet18,cifar10_noisy_label,SphericalScore,outer outer
133,ZeroOneScore TotalRisk outer central,BrierScore,"[0.8292069563340282, 0.8203730465301489, 0.740...",resnet18,cifar10_noisy_label,ZeroOneScore,outer central
186,ZeroOneScore BayesRisk central,CrossEntropy,"[0.7820142650723241, 0.837232767292478, 0.8115...",resnet18,cifar10_noisy_label,ZeroOneScore,central
146,ZeroOneScore TotalRisk central outer,SphericalScore,"[0.8507520993063161, 0.8363808616492008, 0.837...",resnet18,cifar10_noisy_label,ZeroOneScore,central outer
121,BrierScore BayesRisk inner,BrierScore,"[0.8281574686823305, 0.7996790886607885, 0.756...",resnet18,cifar10_noisy_label,BrierScore,inner
253,LogScore energy outer,BrierScore,"[0.79704163546705, 0.7726924115938734, 0.73440...",resnet18,cifar10_noisy_label,LogScore,energy outer
173,ZeroOneScore ExcessRisk central outer,SphericalScore,"[0.8733727796561741, 0.8271212460849693, 0.852...",resnet18,cifar10_noisy_label,ZeroOneScore,central outer
135,ZeroOneScore TotalRisk inner outer,CrossEntropy,"[0.7967131983032528, 0.8441712524894938, 0.808...",resnet18,cifar10_noisy_label,ZeroOneScore,inner outer
120,BrierScore BayesRisk inner,CrossEntropy,"[0.7371669151970955, 0.8334137516389305, 0.836...",resnet18,cifar10_noisy_label,BrierScore,inner


In [4]:
full_mis_rocauc.training_dataset.unique()

array(['cifar10_noisy_label'], dtype=object)

In [5]:
np.hstack(full_mis_rocauc.RocAucScores_array.values).min()

0.48775359511468824

In [6]:
def selector(
    df,
    architecture,
    UQMetric,
):
    arr = np.array(
        df[
            (df.UQMetric == UQMetric)
            & (df.architecture == architecture)
        ].RocAucScores_array.values[0]
    )
    # print(arr)

    # return f"Mean: {arr.mean()}, Std: {arr.std()}"
    return float(arr.mean()), float(arr.std())

In [7]:
def get_specific_stats(
    architecture_,
    loss_function_,
    base_rule_,
):
    selected_results = full_mis_rocauc[
        (full_mis_rocauc.base_rule == base_rule_)
        & (full_mis_rocauc.LossFunction == loss_function_)
    ]

    full_res = {}
    res_dict = {}
    for uqmetric_name in [
        el for el in full_mis_rocauc.UQMetric.unique() if el.startswith(base_rule_)
    ]:
        mean, std = selector(
            df=selected_results,
            UQMetric=uqmetric_name,
            architecture=architecture_,
        )
        res_dict[uqmetric_name] = {"mean": mean, "std": std}
    full_res = res_dict

    return full_res

In [8]:
# full_mis_rocauc['training_dataset'].unique()

In [9]:
architecture = "resnet18"

In [10]:
ce_full_res = get_specific_stats(
    architecture_=architecture,
    loss_function_=LossName.CROSS_ENTROPY.value,
    base_rule_=GName.LOG_SCORE.value,
)

pd.DataFrame.from_dict(ce_full_res).T

,mean,std
LogScore TotalRisk outer outer,0.757266,0.046807
LogScore TotalRisk outer inner,0.783553,0.047686
LogScore TotalRisk outer central,0.762649,0.050384
LogScore TotalRisk inner outer,0.757266,0.046807
LogScore TotalRisk inner inner,0.783553,0.047686
LogScore TotalRisk inner central,0.762649,0.050384
LogScore TotalRisk central outer,0.793767,0.038486
LogScore TotalRisk central inner,0.802061,0.039641
LogScore TotalRisk central central,0.804374,0.036907
LogScore ExcessRisk outer outer,0.683930,0.060733


In [11]:
ce_full_res = get_specific_stats(
    architecture_=architecture,
    loss_function_=LossName.CROSS_ENTROPY.value,
    base_rule_=GName.BRIER_SCORE.value,
)

pd.DataFrame.from_dict(ce_full_res).T

,mean,std
BrierScore TotalRisk outer outer,0.788128,0.036038
BrierScore TotalRisk outer inner,0.796245,0.038219
BrierScore TotalRisk outer central,0.796245,0.038219
BrierScore TotalRisk inner outer,0.788128,0.036038
BrierScore TotalRisk inner inner,0.796245,0.038220
BrierScore TotalRisk inner central,0.796245,0.038220
BrierScore TotalRisk central outer,0.788128,0.036038
BrierScore TotalRisk central inner,0.796245,0.038220
BrierScore TotalRisk central central,0.796245,0.038220
BrierScore ExcessRisk outer outer,0.720286,0.036689


In [12]:
ce_full_res = get_specific_stats(
    architecture_=architecture,
    loss_function_=LossName.CROSS_ENTROPY.value,
    base_rule_=GName.ZERO_ONE_SCORE.value,
)

ce_zo = pd.DataFrame.from_dict(ce_full_res).T

ce_zo

,mean,std
ZeroOneScore TotalRisk outer outer,0.816813,0.016673
ZeroOneScore TotalRisk outer inner,0.809268,0.018199
ZeroOneScore TotalRisk outer central,0.809268,0.018199
ZeroOneScore TotalRisk inner outer,0.816813,0.016673
ZeroOneScore TotalRisk inner inner,0.809268,0.018199
ZeroOneScore TotalRisk inner central,0.809268,0.018199
ZeroOneScore TotalRisk central outer,0.816813,0.016673
ZeroOneScore TotalRisk central inner,0.809268,0.018199
ZeroOneScore TotalRisk central central,0.809268,0.018199
ZeroOneScore ExcessRisk outer outer,0.764256,0.030609


In [50]:
full_mean_tab = None
full_std_tab = None

full_index = None

for ind_dataset, model_source in [
    # (DatasetName.CIFAR10.value, ModelSource.OUR_MODELS.value),
    # (DatasetName.CIFAR100.value, ModelSource.OUR_MODELS.value),
    # (DatasetName.CIFAR10_NOISY_LABEL.value, ModelSource.OUR_MODELS.value),
    # (DatasetName.CIFAR100_NOISY_LABEL.value, ModelSource.OUR_MODELS.value),
    (DatasetName.TINY_IMAGENET.value, ModelSource.TORCH_UNCERTAINTY.value),
]:

    full_mis_rocauc = pd.read_pickle(
        f"{REPOSITORY_ROOT}/tables/central_tables/final/{ind_dataset}_{model_source}_full_mis_rocauc.pkl"
    )
    
    # for (loss_function_, base_rule_) in [
    #     # (LossName.CROSS_ENTROPY.value, GName.LOG_SCORE.value),
    #     # (LossName.BRIER_SCORE.value, GName.BRIER_SCORE.value),
    #     (LossName.SPHERICAL_SCORE.value, GName.SPHERICAL_SCORE.value),
    # ]:
    for (loss_function_, base_rule_) in [
        (LossName.CROSS_ENTROPY.value, GName.LOG_SCORE.value),
        (LossName.CROSS_ENTROPY.value, GName.BRIER_SCORE.value),
        (LossName.CROSS_ENTROPY.value, GName.SPHERICAL_SCORE.value),
        (LossName.CROSS_ENTROPY.value, GName.ZERO_ONE_SCORE.value),
    ]:
        print(loss_function_, base_rule_)
        ce_full_res = get_specific_stats(
            architecture_=architecture,
            loss_function_=loss_function_,
            base_rule_=base_rule_,
        )
        ce_full_res = {ind_dataset: ce_full_res}
        
        pd.DataFrame.from_dict(ce_full_res).sort_index()
        
        # df_aux = pd.DataFrame.from_dict(ce_full_res).loc[
        #     [
        #     f"{base_rule_} BayesRisk outer",
        #     f"{base_rule_} BayesRisk inner",
        #     f"{base_rule_} BayesRisk central",
        #     f"{base_rule_} ExcessRisk outer outer",
        #     f"{base_rule_} ExcessRisk outer inner",
        #     f"{base_rule_} ExcessRisk inner outer",
        #     f"{base_rule_} ExcessRisk outer central",
        #     f"{base_rule_} ExcessRisk central outer",
        #     f"{base_rule_} TotalRisk outer outer",
        #     f"{base_rule_} TotalRisk outer inner",
        #     f"{base_rule_} TotalRisk outer central",
        #     f"{base_rule_} TotalRisk central outer",
        #     ]
        # ]
        
        df_aux = pd.DataFrame.from_dict(ce_full_res).sort_index()
        
        mean_tab = 100 * df_aux.applymap(lambda x: x['mean']).round(4)
        std_tab = 100 * df_aux.applymap(lambda x: x['std']).round(4)

        if (loss_function_ == LossName.CROSS_ENTROPY.value) and (base_rule_ == GName.LOG_SCORE.value):
            full_index = mean_tab.index
    
        if full_mean_tab is None:
            # full_mean_tab = mean_tab
            # full_std_tab = std_tab
            full_mean_tab = mean_tab.reset_index().drop(columns=['index'])
            full_std_tab = std_tab.reset_index().drop(columns=['index'])
        else:
            # full_mean_tab = pd.DataFrame.join(full_mean_tab, mean_tab)
            # full_std_tab = pd.DataFrame.join(full_std_tab, std_tab)

            full_mean_tab= pd.concat([full_mean_tab, mean_tab.reset_index().drop(columns=['index'])], axis=1, ignore_index=True)
            full_std_tab= pd.concat([full_std_tab, std_tab.reset_index().drop(columns=['index'])], axis=1, ignore_index=True)

CrossEntropy LogScore
CrossEntropy BrierScore
CrossEntropy SphericalScore
CrossEntropy ZeroOneScore


/tmp/ipykernel_1729049/4265140004.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mean_tab = 100 * df_aux.applymap(lambda x: x['mean']).round(4)
/tmp/ipykernel_1729049/4265140004.py:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  std_tab = 100 * df_aux.applymap(lambda x: x['std']).round(4)
/tmp/ipykernel_1729049/4265140004.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mean_tab = 100 * df_aux.applymap(lambda x: x['mean']).round(4)
/tmp/ipykernel_1729049/4265140004.py:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  std_tab = 100 * df_aux.applymap(lambda x: x['std']).round(4)
/tmp/ipykernel_1729049/4265140004.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mean_tab = 100 * df_aux.applymap(lambda x: x['mean']).round(4)
/tmp/ipykernel_1729049/4265140004.py:59: FutureWarning: DataFram

In [52]:
# full_mean_tab.index = full_index
# full_std_tab.index = full_index

In [53]:
full_mean_tab = full_mean_tab.sort_index()
full_mean_tab

,0,1,2,3
LogScore BayesRisk central,86.55,86.80,85.83,87.23
LogScore BayesRisk inner,85.50,86.80,86.80,87.23
LogScore BayesRisk outer,84.99,85.89,85.76,85.88
LogScore ExcessRisk central central,50.00,50.00,50.00,50.00
LogScore ExcessRisk central inner,71.77,50.00,81.82,50.00
LogScore ExcessRisk central outer,82.74,75.91,83.60,73.59
LogScore ExcessRisk inner central,66.83,50.00,78.32,50.00
LogScore ExcessRisk inner inner,50.00,50.00,50.00,50.00
LogScore ExcessRisk inner outer,80.18,75.91,79.62,73.59
LogScore ExcessRisk outer central,79.25,75.91,80.73,79.52


In [54]:
full_std_tab = full_std_tab.sort_index()
full_std_tab

,0,1,2,3
LogScore BayesRisk central,0.26,0.28,0.36,0.35
LogScore BayesRisk inner,0.23,0.28,0.28,0.35
LogScore BayesRisk outer,0.24,0.27,0.26,0.27
LogScore ExcessRisk central central,0.00,0.00,0.00,0.00
LogScore ExcessRisk central inner,0.13,0.00,0.47,0.00
LogScore ExcessRisk central outer,0.34,0.59,0.47,0.44
LogScore ExcessRisk inner central,0.11,0.00,0.58,0.00
LogScore ExcessRisk inner inner,0.00,0.00,0.00,0.00
LogScore ExcessRisk inner outer,0.31,0.59,0.54,0.44
LogScore ExcessRisk outer central,0.25,0.59,0.54,0.68


In [55]:
# Merge the mean and std tables based on their index to create a combined LaTeX table.
merged_table = full_mean_tab.round(3).copy()

# For each numeric column, combine the mean and std in the format: mean \pm std
for col in full_mean_tab.columns:
    merged_table[col] = full_mean_tab.round(3)[col].astype(str) + " $\\pm$ " + full_std_tab[col].round(3).astype(str)

# Create LaTeX format
latex_table = merged_table.to_latex(index=True, escape=False, float_format="%.2f")

# Output the resulting LaTeX table for the user
print(latex_table)

\begin{tabular}{lllll}
\toprule
 & 0 & 1 & 2 & 3 \\
\midrule
LogScore BayesRisk central & 86.55 $\pm$ 0.26 & 86.8 $\pm$ 0.28 & 85.83 $\pm$ 0.36 & 87.23 $\pm$ 0.35 \\
LogScore BayesRisk inner & 85.5 $\pm$ 0.23 & 86.8 $\pm$ 0.28 & 86.8 $\pm$ 0.28 & 87.23 $\pm$ 0.35 \\
LogScore BayesRisk outer & 84.99 $\pm$ 0.24 & 85.89 $\pm$ 0.27 & 85.76 $\pm$ 0.26 & 85.88 $\pm$ 0.27 \\
LogScore ExcessRisk central central & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 \\
LogScore ExcessRisk central inner & 71.77 $\pm$ 0.13 & 50.0 $\pm$ 0.0 & 81.82 $\pm$ 0.47 & 50.0 $\pm$ 0.0 \\
LogScore ExcessRisk central outer & 82.74 $\pm$ 0.34 & 75.91 $\pm$ 0.59 & 83.6 $\pm$ 0.47 & 73.59 $\pm$ 0.44 \\
LogScore ExcessRisk inner central & 66.83 $\pm$ 0.11 & 50.0 $\pm$ 0.0 & 78.32 $\pm$ 0.58 & 50.0 $\pm$ 0.0 \\
LogScore ExcessRisk inner inner & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 \\
LogScore ExcessRisk inner outer & 80.18 $\pm$ 0.31 & 75.91 $\pm$ 0.59 & 79.62 $\pm$ 0.54 

In [56]:
replace_dictionary = {
    "BayesRisk central": r"\(\Rtildebayes^{(3)}\)",
    "BayesRisk inner": r"\(\Rtildebayes^{(2)}\)",
    "BayesRisk outer": r"\(\Rtildebayes^{(1)}\)",
    "ExcessRisk central central": r"\(\Rtildeexc^{(3, 3)}\)",
    "ExcessRisk central inner": r"\(\Rtildeexc^{(3, 2)}\)",
    "ExcessRisk central outer": r"\(\Rtildeexc^{(3, 1)}\)",
    "ExcessRisk inner central": r"\(\Rtildeexc^{(2, 3)}\)",
    "ExcessRisk inner inner": r"\(\Rtildeexc^{(2, 2)}\)",
    "ExcessRisk inner outer": r"\(\Rtildeexc^{(2, 1)}\)",
    "ExcessRisk outer central": r"\(\Rtildeexc^{(1, 3)}\)",
    "ExcessRisk outer inner": r"\(\Rtildeexc^{(1, 2)}\)",
    "ExcessRisk outer outer": r"\(\Rtildeexc^{(1, 1)}\)",
    "TotalRisk central central": r"\(\Rtildetot^{(3, 3)}\)",
    "TotalRisk central inner": r"\(\Rtildetot^{(3, 2)}\)",
    "TotalRisk central outer": r"\(\Rtildetot^{(3, 1)}\)",
    "TotalRisk inner central": r"\(\Rtildetot^{(2, 3)}\)",
    "TotalRisk inner inner": r"\(\Rtildetot^{(2, 2)}\)",
    "TotalRisk inner outer": r"\(\Rtildetot^{(2, 1)}\)",
    "TotalRisk outer central": r"\(\Rtildetot^{(1, 3)}\)",
    "TotalRisk outer inner": r"\(\Rtildetot^{(1, 2)}\)",
    "TotalRisk outer outer": r"\(\Rtildetot^{(1, 1)}\)",
    "energy inner": r"\( E(x;\E_{\theta}f_{\theta}) \)",
    "energy outer": r"\( \E_{\theta} E(x;f_{\theta}) \)",
    
} 
for key in replace_dictionary.keys():
    latex_table = latex_table.replace(key, replace_dictionary[key])

print(latex_table)

\begin{tabular}{lllll}
\toprule
 & 0 & 1 & 2 & 3 \\
\midrule
LogScore \(\Rtildebayes^{(3)}\) & 86.55 $\pm$ 0.26 & 86.8 $\pm$ 0.28 & 85.83 $\pm$ 0.36 & 87.23 $\pm$ 0.35 \\
LogScore \(\Rtildebayes^{(2)}\) & 85.5 $\pm$ 0.23 & 86.8 $\pm$ 0.28 & 86.8 $\pm$ 0.28 & 87.23 $\pm$ 0.35 \\
LogScore \(\Rtildebayes^{(1)}\) & 84.99 $\pm$ 0.24 & 85.89 $\pm$ 0.27 & 85.76 $\pm$ 0.26 & 85.88 $\pm$ 0.27 \\
LogScore \(\Rtildeexc^{(3, 3)}\) & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 \\
LogScore \(\Rtildeexc^{(3, 2)}\) & 71.77 $\pm$ 0.13 & 50.0 $\pm$ 0.0 & 81.82 $\pm$ 0.47 & 50.0 $\pm$ 0.0 \\
LogScore \(\Rtildeexc^{(3, 1)}\) & 82.74 $\pm$ 0.34 & 75.91 $\pm$ 0.59 & 83.6 $\pm$ 0.47 & 73.59 $\pm$ 0.44 \\
LogScore \(\Rtildeexc^{(2, 3)}\) & 66.83 $\pm$ 0.11 & 50.0 $\pm$ 0.0 & 78.32 $\pm$ 0.58 & 50.0 $\pm$ 0.0 \\
LogScore \(\Rtildeexc^{(2, 2)}\) & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 & 50.0 $\pm$ 0.0 \\
LogScore \(\Rtildeexc^{(2, 1)}\) & 80.18 $\pm$ 0.31 & 75.91 $\pm$ 0.59 & 79

In [ ]:
full_mis_rocauc = full_mis_rocauc[~full_mis_rocauc.UQMetric.str.endswith("Inner Inner")]

full_mis_rocauc.loc[
    (full_mis_rocauc.RiskType == "Bayes")
    & full_mis_rocauc.UQMetric.str.endswith("Outer"),
    "RiskType",
] = "Bayes Outer"
full_mis_rocauc.loc[
    (full_mis_rocauc.RiskType == "Bayes")
    & full_mis_rocauc.UQMetric.str.endswith("Inner"),
    "RiskType",
] = "Bayes Inner"

full_mis_rocauc.loc[
    (full_mis_rocauc.RiskType == "Total")
    & full_mis_rocauc.UQMetric.str.endswith("Outer"),
    "RiskType",
] = "Total Outer"
full_mis_rocauc.loc[
    (full_mis_rocauc.RiskType == "Total")
    & full_mis_rocauc.UQMetric.str.endswith("Inner"),
    "RiskType",
] = "Total Inner"

In [ ]:
# trunc_df = full_mis_rocauc[
# ~full_mis_rocauc.RiskType.isin(['Bias', 'MV', 'MVBI', 'BiasBI', 'Bregman Information', 'Reverse Bregman Information'])
# # full_mis_rocauc.base_rule.isin(['Brier', 'Logscore', 'Spherical']) &
# # full_mis_rocauc.LossFunction.isin(['Brier', 'Logscore', 'Spherical']) &
# # ~(np.isclose(full_mis_rocauc.RocAucScore, np.float64(0.5)))
# ]

# # trunc_df.sort_values(by='RocAucScore')

# trunc_df.to_csv(os.path.join('tables', 'full_mis_rocauc_only_risks.csv'), index=False)

In [ ]:
grouped_df = extract_same_different_dataframes(
    dataframe_=full_mis_rocauc,
)

In [ ]:
same_dict, _ = collect_scores_into_dict_miss(
    dataframes_list_=[
        grouped_df.logscore_logscore,
        grouped_df.brier_brier,
        grouped_df.spherical_spherical,
    ],
)
same_df = pd.DataFrame.from_dict(same_dict)

same_agg_df = aggregate_over_measures(
    dataframe_=same_df,
    agg_func_="mean",
    by_=["InD"],
)

In [ ]:
different_dict, _ = collect_scores_into_dict_miss(
    dataframes_list_=[
        grouped_df.logscore_not_logscore,
        grouped_df.brier_not_brier,
        grouped_df.spherical_not_spherical,
    ],
)
different_df = pd.DataFrame.from_dict(different_dict)

different_agg_df = aggregate_over_measures(
    dataframe_=different_df,
    agg_func_="mean",
    by_=["InD"],
)

In [ ]:
all_dict, _ = collect_scores_into_dict_miss(
    dataframes_list_=[
        full_mis_rocauc,
    ],
)
all_df = pd.DataFrame.from_dict(all_dict)

all_agg_df = aggregate_over_measures(
    dataframe_=all_df,
    agg_func_="mean",
    by_=["InD"],
)

In [ ]:
display(all_agg_df)
display(same_agg_df)
display(different_agg_df)

In [ ]:
same_agg_df.index

In [ ]:
def enhance_latex_table(input_latex):
    lines = input_latex.split("\n")
    enhanced_lines = []

    for i, line in enumerate(lines):
        if "\\toprule" in line:
            # Add multicolumn headers
            enhanced_lines.append(line)
            enhanced_lines.append(
                r"\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\"
            )
            enhanced_lines.append(r"\cmidrule(lr){1-2} \cmidrule(lr){3-7}")
            continue

        # Add row coloring
        if "\\midrule" in line:
            enhanced_lines.append(line)
            enhanced_lines.append(r"\rowcolor{gray!10}")
        elif "\\bottomrule" in line:
            enhanced_lines.append(r"\end{tabular}")
        else:
            enhanced_lines.append(line)

    return "\n".join(enhanced_lines)


def get_nice_df(df_):
    df_.index = pd.Index(
        data=[
            "CIFAR10",
            "CIFAR100",
            "Missed class CIFAR10",
            "Noisy CIFAR10",
            "Noisy CIFAR100",
        ],
        name="InD",
    )
    df_.columns = [
        # 'Bayes',
        # 'Excess',
        # 'Total',
        "Bayes(O)",
        "Bayes(I)",
        "Total(O)",
        "Total(I)",
        "BI",
        "RBI",
        "EPBI",
        # 'Bias',
        # 'MV',
        # 'MVBI',
        # 'BiasBI',
    ]
    # df_ = df_[['Bayes', 'Excess', 'Total', 'BI', 'RBI']]
    df_ = (100 * df_).round(2)

    display(df_)

    return df_, df_.to_latex(float_format="%.2f")

In [ ]:
measures = [
    "Bayes Outer",
    "Bayes Inner",
    "Total Outer",
    "Total Inner",
    "Bregman Information",
    "Reverse Bregman Information",
    "Expected Pairwise Bregman Information",
]

# measures = ['Bayes', 'Excess', 'Total', 'Bregman Information', 'Reverse Bregman Information', 'Expected Pairwise Bregman Information']

In [ ]:
nice_same = get_nice_df(same_agg_df.copy()[measures])
print(enhance_latex_table(nice_same[1]))

In [ ]:
nice_different = get_nice_df(different_agg_df.copy()[measures])
print(enhance_latex_table(nice_different[1]))

In [ ]:
print(
    enhance_latex_table(
        pd.concat([nice_same[0], nice_different[0]], axis=1).to_latex(
            float_format="%.2f"
        )
    )
)

In [ ]:
nice_same[0].mean()

In [ ]:
nice_same[0].std()

In [ ]:
nice_different[0].mean()

In [ ]:
nice_different[0].std()

In [ ]:
same_agg_df.eq(same_agg_df.max(axis=1), axis=0)

In [ ]:
different_agg_df.eq(different_agg_df.max(axis=1), axis=0)

In [ ]:
all_agg_df.eq(all_agg_df.max(axis=1), axis=0)

In [ ]:
(same_agg_df - different_agg_df) > 0